In [2]:
import graphene
from graphene.relay import Node
from graphene_mongo import MongoengineConnectionField, MongoengineObjectType


In [14]:
from sagas.tests.graphene.models.models import Department as DepartmentModel
from sagas.tests.graphene.models.models import Employee as EmployeeModel
from sagas.tests.graphene.models.models import Role as RoleModel
from pprint import pprint
import json

class Department(MongoengineObjectType):

    class Meta:
        model = DepartmentModel
        interfaces = (Node,)


class Role(MongoengineObjectType):

    class Meta:
        model = RoleModel
        interfaces = (Node,)


class Employee(MongoengineObjectType):

    class Meta:
        model = EmployeeModel
        interfaces = (Node,)


class Query(graphene.ObjectType):
    node = Node.Field()
    all_employees = MongoengineConnectionField(Employee)
    all_role = MongoengineConnectionField(Role)
    role = graphene.Field(Role)

schema = graphene.Schema(query=Query, types=[Department, Employee, Role])

default_query = '''
{
  allEmployees {
    edges {
      node {
        id,
        name,
        department {
          id,
          name
        },
        role {
          id,
          name
        }
      }
    }
  }
}'''.strip()
result = schema.execute(default_query)
print(json.dumps(result.data, indent=2))

{
  "allEmployees": {
    "edges": [
      {
        "node": {
          "id": "RW1wbG95ZWU6NWU2M2Y1NDc3YjEwZjFkNzM1YzE2ZDEx",
          "name": "Peter",
          "department": {
            "id": "RGVwYXJ0bWVudDo1ZTYzZjU0NzdiMTBmMWQ3MzVjMTZkMGQ=",
            "name": "Engineering"
          },
          "role": {
            "id": "Um9sZTo1ZTYzZjU0NzdiMTBmMWQ3MzVjMTZkMTA=",
            "name": "engineer"
          }
        }
      },
      {
        "node": {
          "id": "RW1wbG95ZWU6NWU2M2Y1NDc3YjEwZjFkNzM1YzE2ZDEy",
          "name": "Roy",
          "department": {
            "id": "RGVwYXJ0bWVudDo1ZTYzZjU0NzdiMTBmMWQ3MzVjMTZkMGQ=",
            "name": "Engineering"
          },
          "role": {
            "id": "Um9sZTo1ZTYzZjU0NzdiMTBmMWQ3MzVjMTZkMTA=",
            "name": "engineer"
          }
        }
      },
      {
        "node": {
          "id": "RW1wbG95ZWU6NWU2M2Y1NDc3YjEwZjFkNzM1YzE2ZDEz",
          "name": "Tracy",
          "department": {
            "i

In [6]:
from mongoengine import connect
from sagas.tests.graphene.models.models import *

# You can connect to a real mongo server instance by your own.
connect('graphene-mongo-example', host='mongomock://localhost', alias='default')


def init_db():
    # Create the fixtures
    engineering = Department(name='Engineering')
    engineering.save()

    hr = Department(name='Human Resources')
    hr.save()

    manager = Role(name='manager')
    manager.save()

    engineer = Role(name='engineer')
    engineer.save()

    peter = Employee(name='Peter', department=engineering, role=engineer)
    peter.save()

    roy = Employee(name='Roy', department=engineering, role=engineer)
    roy.save()

    tracy = Employee(name='Tracy', department=hr, role=manager)
    tracy.save()
    
init_db()